## Measure inference performance of ONNX model on low-resource edge device

Now, we’re going to benchmark a couple of previously created ONNX models on our low-resource edge device.

You will execute this notebook *in a Jupyter container running on an edge device*, not on the general-purpose Chameleon Jupyter environment from which you provision resources.

In [ ]:
import os, time
import numpy as np
import onnxruntime as ort

We’ll define a benchmark function. For convenience (since we don’t need real data for benchmarking) we will use random “fake” samples to evaluate our models’ inference performance.

In [ ]:
def benchmark_session(ort_session):

    ## Benchmark inference latency for single sample

    num_trials = 100  # Number of trials
    input_shape = ort_session.get_inputs()[0].shape  # Get expected input shape
    input_dtype = np.float32  # Adjust dtype as needed
    fixed_shape = (1, *input_shape[1:])  

    # Generate a single dummy sample with random values
    single_sample = np.random.rand(*fixed_shape).astype(input_dtype)

    # Warm-up run
    ort_session.run(None, {ort_session.get_inputs()[0].name: single_sample})

    latencies = []
    for _ in range(num_trials):
        start_time = time.time()
        _ = ort_session.run(None, {ort_session.get_inputs()[0].name: single_sample})
        latencies.append(time.time() - start_time)

    print(f"Inference Latency (single sample, median): {np.percentile(latencies, 50) * 1000:.2f} ms")
    print(f"Inference Latency (single sample, 95th percentile): {np.percentile(latencies, 95) * 1000:.2f} ms")
    print(f"Inference Latency (single sample, 99th percentile): {np.percentile(latencies, 99) * 1000:.2f} ms")
    print(f"Inference Throughput (single sample): {num_trials/np.sum(latencies):.2f} FPS")

Now, let’s evaluate our “baseline” ONNX model:

In [ ]:
onnx_model_path = "models/food11.onnx"
ort_session = ort.InferenceSession(onnx_model_path, providers=['CPUExecutionProvider'])
benchmark_session(ort_session)

the model quantized with dynamic quantization:

In [ ]:
onnx_model_path = "models/food11_quantized_dynamic.onnx"
ort_session = ort.InferenceSession(onnx_model_path, providers=['CPUExecutionProvider'])
benchmark_session(ort_session)

and the model quantized with static quantization, for which we permit up to 0.05 decrease in accuracy:

In [ ]:
onnx_model_path = "models/food11_quantized_aggressive.onnx"
ort_session = ort.InferenceSession(onnx_model_path, providers=['CPUExecutionProvider'])
benchmark_session(ort_session)

When you are done, download the fully executed notebook from the Jupyter container environment for later reference. (Note: because it is an executable file, and you are downloading it from a site that is not secured with HTTPS, you may have to explicitly confirm the download in some browsers.)